In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv("data/ml-1m/movies.dat",sep = "::",header=None,engine="python",encoding="latin-1")
movies.columns = ["MovieID","MovieName","Genre"]

In [3]:
ratings = pd.read_csv("data/ml-1m/ratings.dat",sep = "::",header=None,engine="python",encoding="latin-1")
ratings.columns = ["UserID","MovieID","Rating","Timestamp"]

In [4]:
users = pd.read_csv("data/ml-1m/users.dat",sep = "::",header=None,engine="python",encoding="latin-1")
users.columns = ["UserID","Sex","Age","JobsCode","ZipCode"]

In [5]:
training_set = pd.read_csv("data/ml-100k/u1.base",delimiter="\t")
training_set = np.array(training_set,dtype='int')
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [6]:
test_set = pd.read_csv("data/ml-100k/u1.test",delimiter="\t")
test_set = np.array(test_set,dtype='int')
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

In [7]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))
nb_users,nb_movies

(943, 1682)

In [8]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [9]:
training_set = convert(training_set)
test_set = convert(test_set)

In [10]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [11]:
training_set[training_set==0] = -1
training_set[training_set==1] = 0
training_set[training_set==2] = 0
training_set[training_set>=3] = 1

In [12]:
test_set[test_set==0] = -1
test_set[test_set==1] = 0
test_set[test_set==2] = 0
test_set[test_set>=3] = 1

In [13]:
class RBM:
    def __init__(self,nv,nh,batch_size,epochs):
        """
        nv: number of visible nodes
        nh: number of hidden nodes
        """
        self.W = torch.randn(nh,nv)
        self.a = torch.randn(1,nh)
        self.b = torch.randn(1,nv)
        self.nb_batch = batch_size
        self.nb_epoch = epochs
    
    def sample_h(self,x):
        wx = torch.mm(x,self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    def sample_v(self,y):
        wy = torch.mm(y,self.W)
        activation = wy+ self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    def train(self,v0,vk,ph0,phk):
        self.W+=(torch.mm(v0.t(),ph0)-torch.mm(vk.t(),phk)).t()
        self.b+=torch.sum((v0-vk),0)
        self.a+=torch.sum((ph0-phk),0)
    
    def fit(self,training_set):
        for epoch in range(1, self.nb_epoch + 1):
            train_loss = 0
            s = 0.
            for id_user in range(0, nb_users - batch_size, batch_size):
                vk = training_set[id_user : id_user + batch_size]
                v0 = training_set[id_user : id_user + batch_size]
                ph0,_ = self.sample_h(v0)
                for k in range(10):
                    _,hk = self.sample_h(vk)
                    _,vk = self.sample_v(hk)
                    vk[v0<0] = v0[v0<0]
                phk,_ = self.sample_h(vk)
                self.train(v0, vk, ph0, phk)
                train_loss += np.sqrt(torch.mean((v0[v0>=0] - vk[v0>=0])**2))
                s += 1.
            print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
    
    def test(self,training_set,test_set):
        test_loss = 0
        s = 0.
        for id_user in range(nb_users):
            v = training_set[id_user:id_user+1]
            vt = test_set[id_user:id_user+1]
            if len(vt[vt>=0]) > 0:
                _,h = rbm.sample_h(v)
                _,v = rbm.sample_v(h)
                test_loss += np.sqrt(torch.mean((vt[vt>=0] - v[vt>=0])**2))
                s += 1.
        print('test loss: '+str(test_loss/s))

In [14]:
nv = len(training_set[0])
nh = 100
batch_size = 100
epochs = 10

In [15]:
rbm = RBM(nv,nh,batch_size,epochs)
rbm.fit(training_set)

epoch: 1 loss: tensor(0.5837)
epoch: 2 loss: tensor(0.4857)
epoch: 3 loss: tensor(0.4964)
epoch: 4 loss: tensor(0.4985)
epoch: 5 loss: tensor(0.4984)
epoch: 6 loss: tensor(0.4963)
epoch: 7 loss: tensor(0.4970)
epoch: 8 loss: tensor(0.4970)
epoch: 9 loss: tensor(0.4982)
epoch: 10 loss: tensor(0.4958)


In [16]:
rbm.test(training_set,test_set)

test loss: tensor(0.4759)
